# Note: This script will require both Python and R kernals

# Setting up the environment

## Setting up python and R environment, and loading python packages 

In [1]:
import pandas as pd # load and manipulate data for One-Hot ncoding
import numpy as np # calculate the mean and std. dev. 
import matplotlib.pyplot as plt # to plot figures
plt.style.use("fivethirtyeight")
# !pip install --upgrade jedi==0.17.2
# !pip install jedi
## Setting up environments to run R in Jupyter

import os
os.environ['R_HOME'] = r'C:\Program Files\R\R-4.4.1'

# os.environ['X13_PATH'] = r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\x13as_html-v1-1-b60\x13as'
os.environ['X13_PATH'] = r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\x13as_html-v1-1-b60\x13as'

x13path = None
x13path = r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\x13as_html-v1-1-b60\x13as'

r_home = os.environ.get("R_HOME")
print('R_HOME:', r_home)

x13_path = os.environ.get("X13_PATH")
print('X13_PATHs:', x13_path, ' | ', x13path)

## Setting up R

print('Set up RPY2')


!pip install rpy2 --quiet
import rpy2
import rpy2.robjects as ro
import rpy2.rinterface as ri
from rpy2.robjects import r

# Extra RPY2 items

from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
from rpy2.robjects.vectors import IntVector, FloatVector
import rpy2.ipython.html
import rpy2.robjects.lib.ggplot2 as gp
from rpy2.ipython.ggplot import image_png
import rpy2.rinterface as rinterface

#rinterface.initr()
pandas2ri.activate()

%load_ext rpy2.ipython



R_HOME: C:\Program Files\R\R-4.4.1
X13_PATHs: C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\x13as_html-v1-1-b60\x13as  |  C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\x13as_html-v1-1-b60\x13as
Set up RPY2


C:\ProgramData\anaconda3\Lib\site-packages\rpy2\robjects\packages.py:367: UserWarning: The symbol 'quartz' is not in this R namespace/package.
  warnings.warn(
C:\ProgramData\anaconda3\Lib\site-packages\rpy2\robjects\lib\ggplot2.py:70: UserWarning: This was designed against ggplot2 versions starting with 3.3. but you have 3.5.1
  warnings.warn(


## Loading R packages

In [2]:
%%R

# install.packages("seasonal")
# install.packages("seasonalview")

library(reshape2)
library(lubridate)
library(tidyr)
library(shiny)
library(ggplot2)
library(forecast)
library(dplyr)

# install.packages("ggfortify")
library(ggfortify)

# install.packages("Microsoft365R")
suppressPackageStartupMessages(library(Microsoft365R))

# install.packages("strucchange")
suppressPackageStartupMessages(library(strucchange))

suppressPackageStartupMessages(library(tseries))
suppressPackageStartupMessages(library(openxlsx))

## Packages to interface with x13-SEATS
library(seasonal)
library(seasonalview)
suppressPackageStartupMessages(library(readr))
## Check seasonal library
checkX13()


Attaching package: 'lubridate'

The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Attaching package: 'tidyr'

The following object is masked from 'package:reshape2':

    smiths

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 
This is forecast 8.23.0 
  Need help getting started? Try the online textbook FPP:
  http://otexts.com/fpp2/

Attaching package: 'dplyr'

The following objects are masked from 'package:stats':

    filter, lag

The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union

Registered S3 methods overwritten by 'ggfortify':
  method                 from    
  autoplot.Arima         forecast
  autoplot.acf           forecast
  autoplot.ar            forecast
  autoplot.bats          forecast
  autoplot.decomposed.ts forecast
  autoplot.ets           forecast
  autoplot.forecast      forecast
  autoplot.stl           forecast
  autoplot.ts   

# Step 01: Preparing the dataset

## Preprocessing the dependent variable: nm_oil_price

### - Loading the data

In [3]:
# nm_oil_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'nm_oil_price'))
nm_oil_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'nm_oil_price'))

nm_oil_price.rename(columns = {"Month": "Date", "New Mexico Crude Oil First Purchase Price Dollars per Barrel": "nm_oil_price"}, inplace = True)
nm_oil_price['Date'] = pd.to_datetime(nm_oil_price['Date'])
nm_oil_price = nm_oil_price.set_index('Date')

### - Finding quarterly average from monthly data

In [4]:
nm_oil_price = nm_oil_price.resample('QS').mean()

### - Creating a sample of twenty years period

In [5]:
nm_oil_price = nm_oil_price['2001-01-01' : '2023-06-30']
# nm_oil_price_r = nm_oil_price.reset_index()
print(nm_oil_price)

            nm_oil_price
Date                    
2001-01-01     27.215626
2001-04-01     25.243032
2001-07-01     25.139517
2001-10-01     18.382252
2002-01-01     19.688715
...                  ...
2022-04-01    109.638961
2022-07-01     93.402468
2022-10-01     82.709246
2023-01-01     74.988884
2023-04-01     74.403726

[90 rows x 1 columns]


In [6]:
%%R -i nm_oil_price

nm_oil_price_r <- nm_oil_price
nm_oil_price_r <- ts(nm_oil_price_r, start = 2001, end = 2024, frequency =4)
nm_oil_price_r <-  as.data.frame(as_tibble(seas(nm_oil_price_r, x11= "",)))
nm_oil_price_r <-  nm_oil_price_r[-c(3:7)]
 colnames(nm_oil_price_r)[1] = "Date"
colnames(nm_oil_price_r)[2] = "nm_oil_price_sea"
print(nm_oil_price_r)

         Date nm_oil_price_sea
1  2001-01-01         27.16088
2  2001-04-01         25.08883
3  2001-07-01         23.90018
4  2001-10-01         19.76064
5  2002-01-01         19.70143
6  2002-04-01         24.59471
7  2002-07-01         25.53010
8  2002-10-01         27.35501
9  2003-01-01         32.77590
10 2003-04-01         27.69014
11 2003-07-01         27.07028
12 2003-10-01         30.61430
13 2004-01-01         33.98137
14 2004-04-01         36.36277
15 2004-07-01         39.59140
16 2004-10-01         45.96022
17 2005-01-01         47.73334
18 2005-04-01         48.14248
19 2005-07-01         57.60111
20 2005-10-01         56.87974
21 2006-01-01         60.86754
22 2006-04-01         64.94235
23 2006-07-01         64.35854
24 2006-10-01         56.00608
25 2007-01-01         57.37306
26 2007-04-01         58.29867
27 2007-07-01         70.53957
28 2007-10-01         87.75160
29 2008-01-01         97.86608
30 2008-04-01        118.42174
31 2008-07-01        114.05543
32 2008-

## Preprocessing the predictors: wti_spot_price


### - Loading the data

In [7]:
# wti_spot_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'wti_spot'))
wti_spot_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'wti_spot'))
wti_spot_price.rename(columns = {"Month": "Date", "Cushing OK WTI Spot Price FOB Dollars per Barrel": "wti_spot_price"}, inplace = True)
wti_spot_price['Date'] = pd.to_datetime(wti_spot_price['Date'])
wti_spot_price = wti_spot_price.set_index('Date')

### - Finding quarterly average from monthly data

In [8]:
wti_spot_price = wti_spot_price.resample('QS').mean()

### - Creating a sample of twenty years period

In [9]:
wti_spot_price = wti_spot_price['2001-01-01' : '2023-06-30']
print(wti_spot_price)

            wti_spot_price
Date                      
2001-01-01       28.816667
2001-04-01       27.906667
2001-07-01       26.666667
2001-10-01       20.400000
2002-01-01       21.656667
...                    ...
2022-04-01      108.723333
2022-07-01       93.183333
2022-10-01       82.786667
2023-01-01       76.076667
2023-04-01       73.760000

[90 rows x 1 columns]


In [10]:
%%R -i wti_spot_price

wti_spot_price_r <- ts(wti_spot_price, start = 2001, end = 2024, frequency =4)
wti_spot_price_r <-  as.data.frame(as_tibble(seas(wti_spot_price_r, x11= "",)))
wti_spot_price_r <-  wti_spot_price_r[-c(3:7)]
colnames(wti_spot_price_r)[1] = "Date"
colnames(wti_spot_price_r)[2] = "wti_spot_price_sea"
print(wti_spot_price_r)

         Date wti_spot_price_sea
1  2001-01-01           28.91017
2  2001-04-01           27.69969
3  2001-07-01           25.73033
4  2001-10-01           21.38795
5  2002-01-01           21.81504
6  2002-04-01           26.12985
7  2002-07-01           27.25751
8  2002-10-01           29.14862
9  2003-01-01           34.46813
10 2003-04-01           28.87850
11 2003-07-01           28.87041
12 2003-10-01           32.13041
13 2004-01-01           35.93025
14 2004-04-01           38.20693
15 2004-07-01           42.19873
16 2004-10-01           49.26638
17 2005-01-01           50.95922
18 2005-04-01           52.50230
19 2005-07-01           61.40673
20 2005-10-01           60.82922
21 2006-01-01           65.21794
22 2006-04-01           69.32184
23 2006-07-01           68.66407
24 2006-10-01           60.70217
25 2007-01-01           60.54630
26 2007-04-01           63.23387
27 2007-07-01           74.15048
28 2007-10-01           91.34357
29 2008-01-01          100.49154
30 2008-04

## Preprocessing the predictors: brent_spot_price

### - Loading the data

In [11]:
# brent_spot_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'brent_spot'))
brent_spot_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'brent_spot'))
brent_spot_price.rename(columns = {"Month": "Date", "Europe Brent Spot Price FOB Dollars per Barrel": "brent_spot_price"}, inplace = True)
brent_spot_price['Date'] = pd.to_datetime(brent_spot_price['Date'])
brent_spot_price = brent_spot_price.set_index('Date')

### - Finding quarterly average from monthly data

In [12]:
brent_spot_price = brent_spot_price.resample('QS').mean()

### - Creating a sample of twenty years period

In [13]:
brent_spot_price = brent_spot_price['2001-01-01' : '2023-06-30']
print(brent_spot_price)

            brent_spot_price
Date                        
2001-01-01         25.873333
2001-04-01         27.273333
2001-07-01         25.303333
2001-10-01         19.350000
2002-01-01         21.133333
...                      ...
2022-04-01        113.543333
2022-07-01        100.713333
2022-10-01         88.556667
2023-01-01         81.173333
2023-04-01         78.316667

[90 rows x 1 columns]


In [14]:
%%R -i brent_spot_price

brent_spot_price_r <- ts(brent_spot_price, start = 2001, end = 2024, frequency =4)
brent_spot_price_r <-  as.data.frame(as_tibble(seas(brent_spot_price_r, x11= "",)))
brent_spot_price_r <-  brent_spot_price_r[-c(3:7)]
colnames(brent_spot_price_r)[1] = "Date"
colnames(brent_spot_price_r)[2] = "brent_spot_price_sea"
print(brent_spot_price_r)

         Date brent_spot_price_sea
1  2001-01-01             26.02770
2  2001-04-01             27.07264
3  2001-07-01             24.30893
4  2001-10-01             20.34046
5  2002-01-01             21.36258
6  2002-04-01             24.89942
7  2002-07-01             25.75489
8  2002-10-01             27.79940
9  2003-01-01             31.90913
10 2003-04-01             25.98428
11 2003-07-01             26.96131
12 2003-10-01             30.61387
13 2004-01-01             32.64538
14 2004-04-01             35.00514
15 2004-07-01             39.61705
16 2004-10-01             45.57168
17 2005-01-01             48.96013
18 2005-04-01             50.59146
19 2005-07-01             59.59644
20 2005-10-01             58.39363
21 2006-01-01             63.64901
22 2006-04-01             67.79270
23 2006-07-01             67.83051
24 2006-10-01             61.25641
25 2007-01-01             60.08797
26 2007-04-01             66.13151
27 2007-07-01             73.55882
28 2007-10-01       

## Preprocessing the predictors: dubai_spot_price

### - Loading the data

In [15]:
# dubai_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'Monthly Prices', index_col=None, na_values=['NA'], usecols="A, D"))
dubai_price = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'Monthly Prices', index_col=None, na_values=['NA'], usecols="A, D"))
dubai_price = dubai_price.dropna()
dubai_price.rename(columns = {"Month": "Date", "Crude oil, Dubai": "dubai_price"}, inplace = True)
dubai_price['Date'] = pd.to_datetime(dubai_price['Date'])
dubai_price = dubai_price.set_index('Date')

### - Finding quarterly average from monthly data

In [16]:
dubai_price = dubai_price.resample('QS').mean()

### - Creating a sample of twenty years period

In [17]:
dubai_price = dubai_price['2001-01-01' : '2023-06-30']
print(dubai_price)

           dubai_price
Date                  
2001-01-01   23.620364
2001-04-01   25.095437
2001-07-01   23.904812
2001-10-01   18.238671
2002-01-01   20.019124
...                ...
2022-04-01      108.91
2022-07-01   98.286667
2022-10-01       84.55
2023-01-01   79.586667
2023-04-01       77.86

[90 rows x 1 columns]


In [18]:
%%R -i dubai_price

dubai_price_r <- ts(dubai_price, start = 2001, end = 2024, frequency =4)
dubai_price_r <-  as.data.frame(as_tibble(seas(dubai_price_r, x11= "",)))
dubai_price_r <-  dubai_price_r[-c(3:7)]
colnames(dubai_price_r)[1] = "Date"
colnames(dubai_price_r)[2] = "dubai_price_sea"
print(dubai_price_r)

C:\ProgramData\anaconda3\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "dubai_price". Fall back to string conversion. The error is: <class 'float'>
  warnings.warn('Error while trying to convert '


         Date dubai_price_sea
1  2001-01-01      22.0001840
2  2001-04-01      25.3566946
3  2001-07-01      20.7808288
4  2001-10-01      19.8391205
5  2002-01-01      21.0543400
6  2002-04-01      22.3904462
7  2002-07-01      25.5716118
8  2002-10-01      26.0100838
9  2003-01-01      31.1104777
10 2003-04-01      23.4320024
11 2003-07-01      26.1613164
12 2003-10-01      30.2905316
13 2004-01-01      32.3326955
14 2004-04-01      33.2599947
15 2004-07-01      33.9250846
16 2004-10-01      36.3929162
17 2005-01-01      39.6599277
18 2005-04-01      43.0155200
19 2005-07-01      51.9029275
20 2005-10-01      52.1244013
21 2006-01-01      59.2303933
22 2006-04-01      64.7276442
23 2006-07-01      66.0654297
24 2006-10-01      56.6775940
25 2007-01-01      57.6609395
26 2007-04-01      65.4776345
27 2007-07-01      71.5203596
28 2007-10-01      85.0447301
29 2008-01-01      91.0390373
30 2008-04-01      16.1570692
31 2008-07-01      15.1689435
32 2008-10-01      52.6246208
33 2009-01

## Preprocessing the predictors: index_china_gdp

### - Loading the data

In [19]:
# index_china_gdp = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'index_china_gdp'))
index_china_gdp = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'index_china_gdp'))
index_china_gdp = index_china_gdp.dropna()
index_china_gdp.rename(columns = {"observation_date": "Date", "CHNLORSGPNOSTSAM": "index_china_gdp"}, inplace = True)
index_china_gdp['Date'] = pd.to_datetime(index_china_gdp['Date'])
index_china_gdp = index_china_gdp.set_index('Date')

### - Finding quarterly average from monthly data

In [20]:
index_china_gdp = index_china_gdp.resample('QS').mean()

### - Creating a sample of twenty years period

In [21]:
index_china_gdp = index_china_gdp['2001-01-01' : '2023-06-30']
print(index_china_gdp)

            index_china_gdp
Date                       
2001-01-01        99.107175
2001-04-01        98.834079
2001-07-01        98.512880
2001-10-01        98.223037
2002-01-01        98.160446
...                     ...
2022-04-01       100.330123
2022-07-01       100.053152
2022-10-01        99.643252
2023-01-01        99.174501
2023-04-01        98.793601

[90 rows x 1 columns]


In [22]:
%%R -i index_china_gdp

index_china_gdp_r <- ts(index_china_gdp, start = 2001, end = 2024, frequency =4)
index_china_gdp_r <-  as.data.frame(as_tibble(seas(index_china_gdp_r, x11= "",)))
index_china_gdp_r <-  index_china_gdp_r[-c(3:7)]
colnames(index_china_gdp_r)[1] = "Date"
colnames(index_china_gdp_r)[2] = "index_china_gdp_sea"
print(index_china_gdp_r)

         Date index_china_gdp_sea
1  2001-01-01            99.10295
2  2001-04-01            98.85148
3  2001-07-01            98.52312
4  2001-10-01            98.20110
5  2002-01-01            98.15388
6  2002-04-01            98.41114
7  2002-07-01            98.70964
8  2002-10-01            98.97631
9  2003-01-01            99.22982
10 2003-04-01            99.48665
11 2003-07-01            99.87617
12 2003-10-01           100.28546
13 2004-01-01           100.50444
14 2004-04-01           100.33819
15 2004-07-01           100.08943
16 2004-10-01            99.90192
17 2005-01-01            99.81428
18 2005-04-01            99.86459
19 2005-07-01           100.00384
20 2005-10-01           100.10334
21 2006-01-01           100.16106
22 2006-04-01           100.22191
23 2006-07-01           100.31456
24 2006-10-01           100.53203
25 2007-01-01           100.83370
26 2007-04-01           101.14235
27 2007-07-01           101.45295
28 2007-10-01           101.66033
29 2008-01-01 

## Preprocessing the predictors: index_us_gdp

### - Loading the data
This data was already in quarterly and seasonaly adjusted, so we do not need to calculate the quarterly vale. 

In [23]:
# index_us_gdp = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'index_us_gdpdef'))
index_us_gdp = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'index_us_gdpdef'))
index_us_gdp = index_us_gdp.dropna()
index_us_gdp.rename(columns = {"observation_date": "Date", "GDPDEF": "index_us_gdp"}, inplace = True)
index_us_gdp['Date'] = pd.to_datetime(index_us_gdp['Date'])
index_us_gdp = index_us_gdp.set_index('Date')
index_us_gdp

,index_us_gdp
Date,
2023-04-01,131.453
2023-01-01,130.800
2022-10-01,129.508
2022-07-01,128.269
2022-04-01,126.907
...,...
1948-01-01,12.731
1947-10-01,12.630
1947-07-01,12.326


### - Creating a sample of twenty years period

In [24]:
# index_us_gdp = index_us_gdp['2001-01-01' : '2023-06-30']
# print(index_us_gdp)

index_us_gdp = index_us_gdp.sort_index().loc['2001-01-01' : '2023-06-30']
print(index_us_gdp)


            index_us_gdp
Date                    
2001-01-01        79.204
2001-04-01        79.683
2001-07-01        79.996
2001-10-01        80.245
2002-01-01        80.504
...                  ...
2022-04-01       126.907
2022-07-01       128.269
2022-10-01       129.508
2023-01-01       130.800
2023-04-01       131.453

[90 rows x 1 columns]


In [25]:
%%R -i index_us_gdp

index_us_gdp_r <- ts(index_us_gdp, start = 2001, end = 2024, frequency =4)
index_us_gdp_r <-  as.data.frame(as_tibble(seas(index_us_gdp_r, x11= "",)))
index_us_gdp_r <-  index_us_gdp_r[-c(3:7)]
colnames(index_us_gdp_r)[1] = "Date"
colnames(index_us_gdp_r)[2] = "index_us_gdp_sea"
print(index_us_gdp_r)

         Date index_us_gdp_sea
1  2001-01-01         79.20260
2  2001-04-01         79.68367
3  2001-07-01         80.00731
4  2001-10-01         80.23500
5  2002-01-01         80.50204
6  2002-04-01         80.78383
7  2002-07-01         81.18218
8  2002-10-01         81.62917
9  2003-01-01         82.04377
10 2003-04-01         82.32851
11 2003-07-01         82.80239
12 2003-10-01         83.30083
13 2004-01-01         83.89540
14 2004-04-01         84.57006
15 2004-07-01         85.11079
16 2004-10-01         85.76674
17 2005-01-01         86.46058
18 2005-04-01         87.08495
19 2005-07-01         87.86119
20 2005-10-01         88.58243
21 2006-01-01         89.22227
22 2006-04-01         90.01196
23 2006-07-01         90.60284
24 2006-10-01         90.95783
25 2007-01-01         91.86257
26 2007-04-01         92.47185
27 2007-07-01         92.90080
28 2007-10-01         93.30958
29 2008-01-01         93.68399
30 2008-04-01         94.15580
31 2008-07-01         94.80579
32 2008-

## Preprocessing the predictors: volatility index: index_cboe_volt

### - Loading the data
This data is in daily format, so we will have to calculate the quarterly mean.

In [26]:
# index_cboe_volt = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'index_cboe_volt'))
index_cboe_volt = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'index_cboe_volt'))
index_cboe_volt = index_cboe_volt.dropna()
index_cboe_volt.rename(columns = {"observation_date": "Date", "VIXCLS": "index_cboe_volt"}, inplace = True)
index_cboe_volt['Date'] = pd.to_datetime(index_cboe_volt['Date'])
index_cboe_volt = index_cboe_volt.set_index('Date')

### - Finding quarterly average from daily data

In [27]:
index_cboe_volt = index_cboe_volt.resample('QS').mean()

### - Creating a sample of twenty years period

In [28]:
index_cboe_volt = index_cboe_volt['2001-01-01' : '2023-06-30']
print(index_cboe_volt)

            index_cboe_volt
Date                       
2001-01-01        25.726452
2001-04-01        23.923810
2001-07-01        25.380339
2001-10-01        27.910000
2002-01-01        21.360833
...                     ...
2022-04-01        27.398438
2022-07-01        24.792727
2022-10-01        25.002656
2023-01-01        20.684062
2023-04-01        16.441250

[90 rows x 1 columns]


In [29]:
%%R -i index_cboe_volt

index_cboe_volt_r <- ts(index_cboe_volt, start = 2001, end = 2024, frequency =4)
index_cboe_volt_r <-  as.data.frame(as_tibble(seas(index_cboe_volt_r, x11= "",)))
index_cboe_volt_r <-  index_cboe_volt_r[-c(3:7)]
colnames(index_cboe_volt_r)[1] = "Date"
colnames(index_cboe_volt_r)[2] = "index_cboe_volt_sea"
print(index_cboe_volt_r)

         Date index_cboe_volt_sea
1  2001-01-01            24.99146
2  2001-04-01            25.41004
3  2001-07-01            25.51208
4  2001-10-01            25.18213
5  2002-01-01            24.36980
6  2002-04-01            22.73380
7  2002-07-01            30.21673
8  2002-10-01            33.52772
9  2003-01-01            28.45993
10 2003-04-01            22.19549
11 2003-07-01            20.26079
12 2003-10-01            16.62058
13 2004-01-01            17.27773
14 2004-04-01            16.53594
15 2004-07-01            14.42657
16 2004-10-01            13.78743
17 2005-01-01            13.08729
18 2005-04-01            13.67321
19 2005-07-01            11.86359
20 2005-10-01            11.90898
21 2006-01-01            13.10178
22 2006-04-01            15.01700
23 2006-07-01            12.33013
24 2006-10-01            11.78306
25 2007-01-01            11.68655
26 2007-04-01            14.33425
27 2007-07-01            23.04278
28 2007-10-01            19.57347
29 2008-01-01 

## Preprocessing the predictors: us unemployment rate: us_unemploy

### - Loading the data
This data is in monthly format, so we will have to calculate the quarterly mean.

### - Loading the data
This data is in monthly format, so we will have to calculate the quarterly mean.

In [30]:
# us_unemploy = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'us_unemploy'))
us_unemploy = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'us_unemploy'))
us_unemploy = us_unemploy.dropna()
us_unemploy.rename(columns = {"observation_date": "Date", "UNRATE": "us_unemploy"}, inplace = True)
us_unemploy['Date'] = pd.to_datetime(us_unemploy['Date'])
us_unemploy = us_unemploy.set_index('Date')

### - Finding quarterly average from monthly data

In [31]:
us_unemploy = us_unemploy.resample('QS').mean()

### - Creating a sample of twenty years period

In [32]:
us_unemploy = us_unemploy['2001-01-01' : '2023-06-30']
print(us_unemploy)

            us_unemploy
Date                   
2001-01-01     4.233333
2001-04-01     4.400000
2001-07-01     4.833333
2001-10-01     5.500000
2002-01-01     5.700000
...                 ...
2022-04-01     3.600000
2022-07-01     3.566667
2022-10-01     3.600000
2023-01-01     3.500000
2023-04-01     3.566667

[90 rows x 1 columns]


In [33]:
%%R -i us_unemploy

us_unemploy_r <- ts(us_unemploy, start = 2001, end = 2024, frequency =4)
us_unemploy_r <-  as.data.frame(as_tibble(seas(us_unemploy_r, x11= "",)))
us_unemploy_r <-  us_unemploy_r[-c(3:7)]
colnames(us_unemploy_r)[1] = "Date"
colnames(us_unemploy_r)[2] = "us_unemploy_sea"
print(us_unemploy_r)

         Date us_unemploy_sea
1  2001-01-01        4.242061
2  2001-04-01        4.393135
3  2001-07-01        4.864115
4  2001-10-01        5.462300
5  2002-01-01        5.711201
6  2002-04-01        5.829171
7  2002-07-01        5.762944
8  2002-10-01        5.830142
9  2003-01-01        5.875920
10 2003-04-01        6.134280
11 2003-07-01        6.156287
12 2003-10-01        5.805332
13 2004-01-01        5.704614
14 2004-04-01        5.601847
15 2004-07-01        5.450498
16 2004-10-01        5.412585
17 2005-01-01        5.303281
18 2005-04-01        5.099963
19 2005-07-01        4.979315
20 2005-10-01        4.952608
21 2006-01-01        4.736884
22 2006-04-01        4.632229
23 2006-07-01        4.640522
24 2006-10-01        4.424094
25 2007-01-01        4.505060
26 2007-04-01        4.499449
27 2007-07-01        4.668436
28 2007-10-01        4.793809
29 2008-01-01        5.005500
30 2008-04-01        5.334428
31 2008-07-01        5.996603
32 2008-10-01        6.863606
33 2009-01

## Preprocessing the predictors: Industrial Production: index_ip_mining 
Mining, Quarrying, and Oil and Gas Extraction: Mining (NAICS = 21), Index 2017=100, Quarterly, Seasonally Adjusted

### - Loading the data
This data is already in quarterly format, so we will have to calculate the quarterly mean.

In [34]:
# index_ip_mining = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'index_ip_mining'))
index_ip_mining = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'index_ip_mining'))
index_ip_mining = index_ip_mining.dropna()
index_ip_mining.rename(columns = {"observation_date": "Date", "IPG21SQ": "index_ip_mining"}, inplace = True)
index_ip_mining['Date'] = pd.to_datetime(index_ip_mining['Date'])
index_ip_mining = index_ip_mining.set_index('Date')

### - Creating a sample of twenty years period

In [35]:
index_ip_mining = index_ip_mining.sort_index().loc['2001-01-01' : '2023-06-30']
print(index_ip_mining)

            index_ip_mining
Date                       
2001-01-01          84.4693
2001-04-01          84.9047
2001-07-01          84.0679
2001-10-01          82.9672
2002-01-01          80.8947
...                     ...
2022-04-01         112.4290
2022-07-01         116.1566
2022-10-01         116.0954
2023-01-01         118.1290
2023-04-01         118.2950

[90 rows x 1 columns]


In [36]:
%%R -i index_ip_mining

index_ip_mining_r <- ts(index_ip_mining, start = 2001, end = 2024, frequency =4)
index_ip_mining_r <-  as.data.frame(as_tibble(seas(index_ip_mining_r, x11= "",)))
index_ip_mining_r <-  index_ip_mining_r[-c(3:7)]
colnames(index_ip_mining_r)[1] = "Date"
colnames(index_ip_mining_r)[2] = "index_ip_mining_sea"
print(index_ip_mining_r)

         Date index_ip_mining_sea
1  2001-01-01            84.35081
2  2001-04-01            84.75117
3  2001-07-01            84.18647
4  2001-10-01            83.13724
5  2002-01-01            80.76434
6  2002-04-01            80.52872
7  2002-07-01            80.26341
8  2002-10-01            79.40709
9  2003-01-01            80.32520
10 2003-04-01            79.68471
11 2003-07-01            80.56315
12 2003-10-01            80.79773
13 2004-01-01            80.52436
14 2004-04-01            79.77705
15 2004-07-01            80.25624
16 2004-10-01            80.44933
17 2005-01-01            81.32441
18 2005-04-01            81.10107
19 2005-07-01            78.07904
20 2005-10-01            76.69723
21 2006-01-01            79.83390
22 2006-04-01            80.86221
23 2006-07-01            82.12905
24 2006-10-01            82.11367
25 2007-01-01            81.05970
26 2007-04-01            81.74080
27 2007-07-01            82.16506
28 2007-10-01            82.54686
29 2008-01-01 

## Preprocessing the predictors: Consumer Price Index for China: index_cpi_china 

### - Loading the data
This data is in monthly format, so we calculate the quarterly mean.

In [37]:
# index_cpi_china = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'index_cpi_china'))
index_cpi_china = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'index_cpi_china'))
index_cpi_china = index_cpi_china.dropna()
index_cpi_china.rename(columns = {"observation_date": "Date", "CPALTT01CNM659N": "index_cpi_china"}, inplace = True)
index_cpi_china['Date'] = pd.to_datetime(index_cpi_china['Date'])
index_cpi_china = index_cpi_china.set_index('Date')

### - Finding quarterly average from monthly data

In [38]:
index_cpi_china = index_cpi_china.resample('QS').mean()

### - Creating a sample of twenty years period

In [39]:
index_cpi_china = index_cpi_china['2001-01-01' : '2023-06-30']
print(index_cpi_china)

            index_cpi_china
Date                       
2001-01-01         0.666667
2001-04-01         1.566667
2001-07-01         0.800000
2001-10-01        -0.133333
2002-01-01        -0.600000
...                     ...
2022-04-01         2.233333
2022-07-01         2.666667
2022-10-01         1.833333
2023-01-01         1.266667
2023-04-01         0.100000

[90 rows x 1 columns]


In [40]:
%%R -i index_cpi_china

index_cpi_china_r <- ts(index_cpi_china, start = 2001, end = 2024, frequency =4)
index_cpi_china_r <-  as.data.frame(as_tibble(seas(index_cpi_china_r, x11= "",)))
index_cpi_china_r <-  index_cpi_china_r[-c(3:7)]
colnames(index_cpi_china_r)[1] = "Date"
colnames(index_cpi_china_r)[2] = "index_cpi_china_sea"
print(index_cpi_china_r)

         Date index_cpi_china_sea
1  2001-01-01         0.716451647
2  2001-04-01         1.453864082
3  2001-07-01         0.804454256
4  2001-10-01        -0.073919742
5  2002-01-01        -0.557756056
6  2002-04-01        -1.169446320
7  2002-07-01        -0.754350686
8  2002-10-01        -0.590315600
9  2003-01-01         0.542788125
10 2003-04-01         0.565072155
11 2003-07-01         0.862439740
12 2003-10-01         2.689187871
13 2004-01-01         2.812563783
14 2004-04-01         4.299917619
15 2004-07-01         5.314798507
16 2004-10-01         3.168206709
17 2005-01-01         2.868879936
18 2005-04-01         1.650262887
19 2005-07-01         1.400078386
20 2005-10-01         1.335953445
21 2006-01-01         1.233549163
22 2006-04-01         1.300984381
23 2006-07-01         1.353333643
24 2006-10-01         1.963873181
25 2007-01-01         2.771874226
26 2007-04-01         3.549227286
27 2007-07-01         6.194143366
28 2007-10-01         6.543527879
29 2008-01-01 

## Preprocessing 2 predictors at once: 
opec supply - opec, non-opec supply- non_opec
This data is in quarterly format. 

In [41]:
# two_x = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'two_x', usecols="A:C"))
two_x = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'two_x', usecols="A:C"))
two_x = two_x.dropna()
two_x.rename(columns = {"obs": "Date", "OPEC": "opec", "NOPEC" : "non_opec"}, inplace = True)
two_x['Date'] = pd.to_datetime(two_x['Date'], format="mixed")
two_x = two_x.set_index('Date')

### - Creating a sample of twenty years period

In [42]:
two_x = two_x.sort_index().loc['2001-01-01' : '2023-06-30']
print(two_x)

                    opec      non_opec
Date                                  
2001-01-01  29031.181000  40054.029000
2001-04-01  27976.069000  39483.162667
2001-07-01  28140.325000  39950.500667
2001-10-01  27307.658667  40521.307667
2002-01-01  26221.237000  40685.595667
...                  ...           ...
2019-10-01  29601.000000  58779.000000
2020-01-01  28246.000000  59931.000000
2020-04-01  25578.000000  54756.000000
2020-07-01  23842.000000  55134.000000
2020-10-01  24945.000000  55800.000000

[80 rows x 2 columns]


In [43]:
%%R -i two_x
two_x_opec_r <- two_x[1]
two_x_opec_r <- ts(two_x_opec_r, start = 2001, end = 2024, frequency =4)
two_x_opec_r <-  as.data.frame(as_tibble(seas(two_x_opec_r, x11= "",)))
two_x_opec_r <-  two_x_opec_r[-c(3:7)]
colnames(two_x_opec_r)[1] = "Date"
colnames(two_x_opec_r)[2] = "opec_sea"
print(two_x_opec_r)

         Date opec_sea
1  2001-01-01 29014.43
2  2001-04-01 28384.33
3  2001-07-01 27959.03
4  2001-10-01 27098.50
5  2002-01-01 26212.01
6  2002-04-01 26075.61
7  2002-07-01 26353.96
8  2002-10-01 27097.90
9  2003-01-01 27595.49
10 2003-04-01 27795.62
11 2003-07-01 27399.80
12 2003-10-01 28756.60
13 2004-01-01 29598.87
14 2004-04-01 30034.31
15 2004-07-01 30736.54
16 2004-10-01 30881.80
17 2005-01-01 31434.74
18 2005-04-01 31912.01
19 2005-07-01 31867.60
20 2005-10-01 31843.65
21 2006-01-01 31659.07
22 2006-04-01 31579.20
23 2006-07-01 31578.21
24 2006-10-01 31083.40
25 2007-01-01 30821.90
26 2007-04-01 30963.64
27 2007-07-01 30909.50
28 2007-10-01 31858.10
29 2008-01-01 32602.45
30 2008-04-01 32695.69
31 2008-07-01 32596.28
32 2008-10-01 31847.04
33 2009-01-01 30350.64
34 2009-04-01 30482.17
35 2009-07-01 30502.29
36 2009-10-01 30755.81
37 2010-01-01 31126.16
38 2010-04-01 31553.84
39 2010-07-01 31625.28
40 2010-10-01 31731.64
41 2011-01-01 31891.54
42 2011-04-01 31261.54
43 2011-07-

In [44]:
%%R -i two_x
two_x_non_opec_r <- two_x[2]
two_x_non_opec_r <- ts(two_x_non_opec_r, start = 2001, end = 2024, frequency =4)
two_x_non_opec_r <-  as.data.frame(as_tibble(seas(two_x_non_opec_r, x11= "",)))
two_x_non_opec_r <-  two_x_non_opec_r[-c(3:7)]
colnames(two_x_non_opec_r)[1] = "Date"
colnames(two_x_non_opec_r)[2] = "non_opec_sea"
print(two_x_non_opec_r)

         Date non_opec_sea
1  2001-01-01     39879.50
2  2001-04-01     39625.16
3  2001-07-01     40144.63
4  2001-10-01     40354.48
5  2002-01-01     40519.73
6  2002-04-01     41034.69
7  2002-07-01     40822.33
8  2002-10-01     40927.07
9  2003-01-01     41162.96
10 2003-04-01     41058.20
11 2003-07-01     41644.02
12 2003-10-01     42048.77
13 2004-01-01     42000.55
14 2004-04-01     42309.99
15 2004-07-01     42194.88
16 2004-10-01     42104.40
17 2005-01-01     41889.15
18 2005-04-01     42368.25
19 2005-07-01     41654.58
20 2005-10-01     41615.59
21 2006-01-01     41651.52
22 2006-04-01     41663.34
23 2006-07-01     41951.46
24 2006-10-01     41909.41
25 2007-01-01     41911.24
26 2007-04-01     41858.03
27 2007-07-01     41676.22
28 2007-10-01     41490.69
29 2008-01-01     41383.70
30 2008-04-01     41404.70
31 2008-07-01     41065.77
32 2008-10-01     41195.06
33 2009-01-01     41431.43
34 2009-04-01     41621.67
35 2009-07-01     42059.84
36 2009-10-01     42024.90
3

## Preprocessing the predictors CPI US: index_cpi_us 
This data is in monthly format. We calculated the quarterly mean. 

### - Loading the data

In [45]:
# index_cpi_us  = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'cpi_us'))
index_cpi_us  = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'cpi_us'))
index_cpi_us  = index_cpi_us.dropna()
index_cpi_us.rename(columns = {"observation_date": "Date", "CPALTT01USM657N": "index_cpi_us",}, inplace = True)
index_cpi_us ['Date'] = pd.to_datetime(index_cpi_us['Date'])
index_cpi_us = index_cpi_us.set_index('Date')

### - Finding quarterly average from monthly data

In [46]:
index_cpi_us = index_cpi_us.resample('QS').mean()

### - Creating a sample of twenty years period

In [47]:
index_cpi_us = index_cpi_us['2001-01-01' : '2023-06-30']
print(index_cpi_us)

            index_cpi_us
Date                    
2001-01-01      0.419829
2001-04-01      0.339444
2001-07-01      0.056602
2001-10-01     -0.299975
2002-01-01      0.394686
...                  ...
2022-04-01      1.011404
2022-07-01      0.055942
2022-10-01     -0.000787
2023-01-01      0.562940
2023-04-01      0.360213

[90 rows x 1 columns]


In [48]:
%%R -i index_cpi_us

index_cpi_us_r <- ts(index_cpi_us, start = 2001, end = 2024, frequency =4)
index_cpi_us_r <-  as.data.frame(as_tibble(seas(index_cpi_us_r, x11= "",)))
index_cpi_us_r <-  index_cpi_us_r[-c(3:7)]
colnames(index_cpi_us_r)[1] = "Date"
colnames(index_cpi_us_r)[2] = "index_cpi_us_sea"
print(index_cpi_us_r)

         Date index_cpi_us_sea
1  2001-01-01      0.098147755
2  2001-04-01      0.313271519
3  2001-07-01      0.096393579
4  2001-10-01      0.008105738
5  2002-01-01      0.073489171
6  2002-04-01      0.173884151
7  2002-07-01      0.255127547
8  2002-10-01      0.280037698
9  2003-01-01      0.288185383
10 2003-04-01     -0.134926073
11 2003-07-01      0.343454698
12 2003-10-01      0.127261523
13 2004-01-01      0.242487052
14 2004-04-01      0.346903537
15 2004-07-01      0.132055402
16 2004-10-01      0.348847038
17 2005-01-01      0.222904600
18 2005-04-01      0.101203036
19 2005-07-01      0.880836190
20 2005-10-01     -0.085314528
21 2006-01-01      0.227371560
22 2006-04-01      0.371104973
23 2006-07-01      0.189727858
24 2006-10-01      0.038599663
25 2007-01-01      0.334895705
26 2007-04-01      0.311723141
27 2007-07-01      0.237425559
28 2007-10-01      0.442396912
29 2008-01-01      0.312517198
30 2008-04-01      0.657729782
31 2008-07-01      0.200012652
32 2008-

## Preprocessing 3 predictors at once: 
Global energy index - index_energy, metal and minerals index- index_metal, precious metal index - index_precious
This data is in monthly format format, so we calculate the quarterly means. 

### - Loading the data

In [49]:
# three_x = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'monthly_indices'))
three_x = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'monthly_indices'))
three_x = three_x.dropna()
three_x['Date'] = pd.to_datetime(three_x['Date'])
three_x = three_x.set_index('Date')
three_x.rename(columns = {"Energy": "index_energy", "Metals  & Minerals": "index_metal", "Precious Metals" : "index_precious"}, inplace = True)

### - Finding quarterly average from monthly data

In [50]:
three_x = three_x.resample('QS').mean()

### - Creating a sample of twenty years period

In [51]:
three_x = three_x['2001-01-01' : '2023-06-30']
print(three_x)

           index_energy index_metal index_precious
Date                                              
2001-01-01    39.908913   37.514824      22.225362
2001-04-01    38.847583   35.850699       22.34024
2001-07-01    34.247092   32.716238      22.427525
2001-10-01    27.180842     31.4506      22.616555
2002-01-01     28.92824   33.278799       23.65152
...                 ...         ...            ...
2022-04-01   163.433933  125.418138     142.282967
2022-07-01    167.56567  100.579267      129.52176
2022-10-01   138.843977   101.53573     131.772869
2023-01-01   111.096934  111.485978     143.121441
2023-04-01   100.448766  103.036955     150.513912

[90 rows x 3 columns]


In [52]:
%%R -i three_x

three_x_index_energy_r <- three_x[1]
three_x_index_energy_r <- ts(three_x_index_energy_r, start = 2001, end = 2024, frequency =4)
three_x_index_energy_r <-  as.data.frame(as_tibble(seas(three_x_index_energy_r, x11= "",)))
three_x_index_energy_r <-  three_x_index_energy_r[-c(3:7)]
colnames(three_x_index_energy_r)[1] = "Date"
colnames(three_x_index_energy_r)[2] = "index_energy_sea"
print(three_x_index_energy_r)

C:\ProgramData\anaconda3\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "index_energy". Fall back to string conversion. The error is: <class 'float'>
  warnings.warn('Error while trying to convert '
C:\ProgramData\anaconda3\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "index_metal". Fall back to string conversion. The error is: <class 'float'>
  warnings.warn('Error while trying to convert '
C:\ProgramData\anaconda3\Lib\site-packages\rpy2\robjects\pandas2ri.py:65: UserWarning: Error while trying to convert the column "index_precious". Fall back to string conversion. The error is: <class 'float'>
  warnings.warn('Error while trying to convert '


         Date index_energy_sea
1  2001-01-01        37.365113
2  2001-04-01        36.369764
3  2001-07-01        31.625120
4  2001-10-01        29.522924
5  2002-01-01        29.581693
6  2002-04-01        31.382044
7  2002-07-01        32.301880
8  2002-10-01        34.560932
9  2003-01-01        42.127864
10 2003-04-01        35.192375
11 2003-07-01        37.635686
12 2003-10-01        40.795516
13 2004-01-01        44.032014
14 2004-04-01        43.738530
15 2004-07-01        44.778853
16 2004-10-01        52.099487
17 2005-01-01        58.325200
18 2005-04-01        59.846850
19 2005-07-01        74.122721
20 2005-10-01        74.325101
21 2006-01-01        75.538602
22 2006-04-01        79.061438
23 2006-07-01        76.682609
24 2006-10-01        70.454311
25 2007-01-01        72.304793
26 2007-04-01        79.450767
27 2007-07-01        83.734652
28 2007-10-01         6.493442
29 2008-01-01        12.328865
30 2008-04-01        23.219768
31 2008-07-01        22.292780
32 2008-

In [53]:
%%R -i three_x

three_x_index_metal_r <- three_x[2]
three_x_index_metal_r <- ts(three_x_index_metal_r, start = 2001, end = 2024, frequency =4)
three_x_index_metal_r <-  as.data.frame(as_tibble(seas(three_x_index_metal_r, x11= "",)))
three_x_index_metal_r <-  three_x_index_metal_r[-c(3:7)]
colnames(three_x_index_metal_r)[1] = "Date"
colnames(three_x_index_metal_r)[2] = "index_metal_sea"
print(three_x_index_metal_r)

         Date index_metal_sea
1  2001-01-01       32.400043
2  2001-04-01       31.455034
3  2001-07-01       27.302007
4  2001-10-01       24.809339
5  2002-01-01       26.429118
6  2002-04-01       28.561953
7  2002-07-01       26.088024
8  2002-10-01       27.841472
9  2003-01-01       29.667868
10 2003-04-01       29.550420
11 2003-07-01       33.668185
12 2003-10-01       35.983144
13 2004-01-01       36.135012
14 2004-04-01       35.322217
15 2004-07-01       38.172438
16 2004-10-01       41.254227
17 2005-01-01       42.771768
18 2005-04-01       39.765434
19 2005-07-01       44.858712
20 2005-10-01       52.466350
21 2006-01-01       58.391742
22 2006-04-01       80.182152
23 2006-07-01       87.752335
24 2006-10-01        4.623511
25 2007-01-01        1.789723
26 2007-04-01       10.598237
27 2007-07-01        8.016671
28 2007-10-01       11.641296
29 2008-01-01       16.705486
30 2008-04-01       16.453611
31 2008-07-01        9.473353
32 2008-10-01       47.518966
33 2009-01

In [54]:
%%R -i three_x

three_x_index_precious_r <- three_x[2]
three_x_index_precious_r <- ts(three_x_index_precious_r, start = 2001, end = 2024, frequency =4)
three_x_index_precious_r <-  as.data.frame(as_tibble(seas(three_x_index_precious_r, x11= "",)))
three_x_index_precious_r <-  three_x_index_precious_r[-c(3:7)]
colnames(three_x_index_precious_r)[1] = "Date"
colnames(three_x_index_precious_r)[2] = "index_precious_sea"
print(three_x_index_precious_r)

         Date index_precious_sea
1  2001-01-01          32.400043
2  2001-04-01          31.455034
3  2001-07-01          27.302007
4  2001-10-01          24.809339
5  2002-01-01          26.429118
6  2002-04-01          28.561953
7  2002-07-01          26.088024
8  2002-10-01          27.841472
9  2003-01-01          29.667868
10 2003-04-01          29.550420
11 2003-07-01          33.668185
12 2003-10-01          35.983144
13 2004-01-01          36.135012
14 2004-04-01          35.322217
15 2004-07-01          38.172438
16 2004-10-01          41.254227
17 2005-01-01          42.771768
18 2005-04-01          39.765434
19 2005-07-01          44.858712
20 2005-10-01          52.466350
21 2006-01-01          58.391742
22 2006-04-01          80.182152
23 2006-07-01          87.752335
24 2006-10-01           4.623511
25 2007-01-01           1.789723
26 2007-04-01          10.598237
27 2007-07-01           8.016671
28 2007-10-01          11.641296
29 2008-01-01          16.705486
30 2008-04

## Preprocessing the predictors non-opec supply change: del_supply_nonopec
This data is in quarterly format

In [55]:
# del_supply_nonopec = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'del_nonopec_supply', usecols= "A, B"))
del_supply_nonopec = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'del_nonopec_supply', usecols= "A, B"))
del_supply_nonopecsupply = del_supply_nonopec.dropna()
del_supply_nonopec['Date'] = pd.to_datetime(del_supply_nonopec['Date'], format = "mixed")
del_supply_nonopec = del_supply_nonopec.set_index('Date')
del_supply_nonopec.rename(columns = {"non-OPEC Production change": "del_supply_nonopec"}, inplace = True)
del_supply_nonopec = del_supply_nonopec.sort_index().loc['2001-01-01' : '2023-06-30']
print(del_supply_nonopec)

            del_supply_nonopec
Date                          
2001-01-01                0.72
2001-04-01                0.48
2001-07-01                0.59
2001-10-01                0.64
2002-01-01                0.80
...                        ...
2019-10-01                1.84
2020-01-01                2.45
2020-04-01               -3.94
2020-07-01               -3.88
2020-10-01               -4.61

[80 rows x 1 columns]


In [56]:
%%R -i del_supply_nonopec

del_supply_nonopec_r <- ts(del_supply_nonopec, start = 2001, end = 2024, frequency =4)
del_supply_nonopec_r <-  as.data.frame(as_tibble(seas(del_supply_nonopec_r, x11= "",)))
del_supply_nonopec_r <-  del_supply_nonopec_r[-c(3:7)]
colnames(del_supply_nonopec_r)[1] = "Date"
colnames(del_supply_nonopec_r)[2] = "del_supply_nonopec_sea"
print(del_supply_nonopec_r)

         Date del_supply_nonopec_sea
1  2001-01-01             0.77528785
2  2001-04-01             0.29816094
3  2001-07-01             0.63696590
4  2001-10-01             0.72142937
5  2002-01-01             1.04616923
6  2002-04-01             1.18307569
7  2002-07-01             0.79476540
8  2002-10-01             0.81639090
9  2003-01-01             1.18224385
10 2003-04-01             0.11818806
11 2003-07-01             1.34572275
12 2003-10-01             1.67945807
13 2004-01-01             1.22318594
14 2004-04-01             1.57036796
15 2004-07-01             0.93398107
16 2004-10-01             0.53923442
17 2005-01-01             0.35810435
18 2005-04-01             0.22870170
19 2005-07-01            -0.27656644
20 2005-10-01            -0.71943420
21 2006-01-01            -0.14012109
22 2006-04-01            -0.53981932
23 2006-07-01             1.25690996
24 2006-10-01             1.05721361
25 2007-01-01             0.59243121
26 2007-04-01             0.51044422
2

## Preprocessing 2 predictors at once: capacity change nonopec, world gdp
These variables are in quarterly format

In [57]:
# two_x_2 = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'capacity', usecols= "A, C, D"))
two_x_2 = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'capacity', usecols= "A, C, D"))
two_x_2 = two_x_2.dropna()
two_x_2['Date'] = pd.to_datetime(two_x_2['Date'], format ="mixed")
two_x_2 = two_x_2.set_index('Date')
two_x_2.rename(columns = {"Capacity percent change": "del_cap_per", "World GDP Change": "del_world_gdp_per"}, inplace = True)
two_x_2 = two_x_2.sort_index().loc['2001-01-01' : '2023-06-30']
print(two_x_2)

            del_cap_per  del_world_gdp_per
Date                                      
2001-01-01         1.15               3.40
2001-04-01         0.85               2.60
2001-07-01         0.98               2.10
2001-10-01         1.04               1.80
2002-01-01         0.84               2.19
...                 ...                ...
2019-10-01         0.58               2.42
2020-01-01         1.45              -1.42
2020-04-01        -4.93              -8.45
2020-07-01        -4.35              -2.09
2020-10-01        -4.69              -0.22

[80 rows x 2 columns]


In [58]:
%%R -i two_x_2

two_x_2_del_cap_per_r <- two_x_2[1]
two_x_2_del_cap_per_r <- ts(two_x_2_del_cap_per_r, start = 2001, end = 2024, frequency =4)
two_x_2_del_cap_per_r <-  as.data.frame(as_tibble(seas(two_x_2_del_cap_per_r, x11= "",)))
two_x_2_del_cap_per_r <-  two_x_2_del_cap_per_r[-c(3:7)]
colnames(two_x_2_del_cap_per_r)[1] = "Date"
colnames(two_x_2_del_cap_per_r)[2] = "del_cap_per_sea"
print(two_x_2_del_cap_per_r)

         Date del_cap_per_sea
1  2001-01-01      1.24730460
2  2001-04-01      0.57184675
3  2001-07-01      1.06450129
4  2001-10-01      1.13819806
5  2002-01-01      0.94195468
6  2002-04-01      1.51270290
7  2002-07-01      0.86150870
8  2002-10-01      0.02551844
9  2003-01-01     -1.99542821
10 2003-04-01     -4.34322813
11 2003-07-01     -1.46314448
12 2003-10-01      0.98804202
13 2004-01-01      3.89147547
14 2004-04-01      5.10578236
15 2004-07-01      3.26281619
16 2004-10-01      1.42677401
17 2005-01-01      1.23053701
18 2005-04-01      1.78738124
19 2005-07-01      1.20443579
20 2005-10-01      0.45855119
21 2006-01-01     -0.03302482
22 2006-04-01     -0.81692154
23 2006-07-01      1.50330210
24 2006-10-01      1.32412069
25 2007-01-01      0.96167952
26 2007-04-01      0.84754452
27 2007-07-01     -0.02565827
28 2007-10-01      0.68873478
29 2008-01-01      1.13949254
30 2008-04-01      0.75122616
31 2008-07-01      0.83772772
32 2008-10-01      0.46615630
33 2009-01

In [59]:
%%R -i two_x_2

two_x_2_del_world_gdp_per_r <- two_x_2[1]
two_x_2_del_world_gdp_per_r <- ts(two_x_2_del_world_gdp_per_r, start = 2001, end = 2024, frequency =4)
two_x_2_del_world_gdp_per_r <-  as.data.frame(as_tibble(seas(two_x_2_del_world_gdp_per_r, x11= "",)))
two_x_2_del_world_gdp_per_r <-  two_x_2_del_world_gdp_per_r[-c(3:7)]
colnames(two_x_2_del_world_gdp_per_r)[1] = "Date"
colnames(two_x_2_del_world_gdp_per_r)[2] = "del_world_gdp_per_sea"
print(two_x_2_del_world_gdp_per_r)

         Date del_world_gdp_per_sea
1  2001-01-01            1.24730460
2  2001-04-01            0.57184675
3  2001-07-01            1.06450129
4  2001-10-01            1.13819806
5  2002-01-01            0.94195468
6  2002-04-01            1.51270290
7  2002-07-01            0.86150870
8  2002-10-01            0.02551844
9  2003-01-01           -1.99542821
10 2003-04-01           -4.34322813
11 2003-07-01           -1.46314448
12 2003-10-01            0.98804202
13 2004-01-01            3.89147547
14 2004-04-01            5.10578236
15 2004-07-01            3.26281619
16 2004-10-01            1.42677401
17 2005-01-01            1.23053701
18 2005-04-01            1.78738124
19 2005-07-01            1.20443579
20 2005-10-01            0.45855119
21 2006-01-01           -0.03302482
22 2006-04-01           -0.81692154
23 2006-07-01            1.50330210
24 2006-10-01            1.32412069
25 2007-01-01            0.96167952
26 2007-04-01            0.84754452
27 2007-07-01           -0.0

## Preprocessing the predictors- change in saudi supply: del_supply_saudi
This data is in quarterly format

In [60]:
# del_supply_saudi = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'saudi_production', usecols= "A, C"))
del_supply_saudi = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'saudi_production', usecols= "A, C"))
del_supply_saudi = del_supply_saudi.dropna()
del_supply_saudi['Date'] = pd.to_datetime(del_supply_saudi['Date'], format ="mixed")
del_supply_saudi = del_supply_saudi.set_index('Date')
del_supply_saudi.rename(columns = {"Saudi Production Change": "del_supply_saudi",}, inplace = True)
del_supply_saudi = del_supply_saudi.sort_index().loc['2001-01-01' : '2023-06-30']
print(del_supply_saudi)

            del_supply_saudi
Date                        
2001-01-01              0.58
2001-04-01             -0.18
2001-07-01             -0.68
2001-10-01             -1.19
2002-01-01             -1.17
...                      ...
2019-10-01             -0.90
2020-01-01             -0.20
2020-04-01             -0.64
2020-07-01             -0.61
2020-10-01             -0.82

[80 rows x 1 columns]


In [61]:
%%R -i del_supply_saudi

del_supply_saudi_r <- ts(del_supply_saudi, start = 2001, end = 2024, frequency =4)
del_supply_saudi_r <-  as.data.frame(as_tibble(seas(del_supply_saudi_r, x11= "",)))
del_supply_saudi_r <-  del_supply_saudi_r[-c(3:7)]
colnames(del_supply_saudi_r)[1] = "Date"
colnames(del_supply_saudi_r)[2] = "del_supply_saudi_sea"
print(del_supply_saudi_r)

         Date del_supply_saudi_sea
1  2001-01-01           0.52685580
2  2001-04-01          -0.29882032
3  2001-07-01          -0.58578668
4  2001-10-01          -1.11894588
5  2002-01-01          -1.21822419
6  2002-04-01          -0.63867547
7  2002-07-01          -0.18042657
8  2002-10-01           0.42734857
9  2003-01-01           1.57766987
10 2003-04-01           1.59930063
11 2003-07-01           0.82671253
12 2003-10-01           0.56260525
13 2004-01-01          -0.32852857
14 2004-04-01          -0.43899046
15 2004-07-01           1.04508558
16 2004-10-01           1.00081845
17 2005-01-01           0.88409753
18 2005-04-01           0.72899695
19 2005-07-01           0.15664386
20 2005-10-01           0.02380846
21 2006-01-01          -0.10491665
22 2006-04-01          -0.43509594
23 2006-07-01          -0.35009831
24 2006-10-01          -0.70904657
25 2007-01-01          -0.77133753
26 2007-04-01          -0.65651454
27 2007-07-01          -0.50175922
28 2007-10-01       

## Preprocessing the predictors-  opec spare capacity: opec_spare_capacity
This data is in quarterly format

In [62]:
# opec_spare_capacity = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'opec_spare_capacity', usecols= "A, C"))
opec_spare_capacity = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'opec_spare_capacity', usecols= "A, C"))
opec_spare_capacity = opec_spare_capacity.dropna()
opec_spare_capacity['Date'] = pd.to_datetime(opec_spare_capacity['Date'], format ="mixed")
opec_spare_capacity = opec_spare_capacity.set_index('Date')
opec_spare_capacity.rename(columns = {"Spare Capacity": "opec_spare_capacity",}, inplace = True)
opec_spare_capacity = opec_spare_capacity.sort_index().loc['2001-01-01' : '2023-06-30']
print(opec_spare_capacity)

            opec_spare_capacity
Date                           
2001-01-01                 3.63
2001-04-01                 4.65
2001-07-01                 4.44
2001-10-01                 5.21
2002-01-01                 6.10
...                         ...
2019-10-01                 2.07
2020-01-01                 2.78
2020-04-01                 5.49
2020-07-01                 6.98
2020-10-01                 6.22

[80 rows x 1 columns]


In [63]:
%%R -i opec_spare_capacity

opec_spare_capacity_r <- ts(opec_spare_capacity, start = 2001, end = 2024, frequency =4)
opec_spare_capacity_r <-  as.data.frame(as_tibble(seas(opec_spare_capacity_r, x11= "",)))
opec_spare_capacity_r <-  opec_spare_capacity_r[-c(3:7)]
colnames(opec_spare_capacity_r)[1] = "Date"
colnames(opec_spare_capacity_r)[2] = "opec_spare_capacity_sea"
print(opec_spare_capacity_r)

         Date opec_spare_capacity_sea
1  2001-01-01               3.8956180
2  2001-04-01               4.4720891
3  2001-07-01               4.4941065
4  2001-10-01               5.0751320
5  2002-01-01               6.4185173
6  2002-04-01               6.4013785
7  2002-07-01               6.0097942
8  2002-10-01               4.4520410
9  2003-01-01               2.0835637
10 2003-04-01               1.3122653
11 2003-07-01               2.3294520
12 2003-10-01               1.9993321
13 2004-01-01               1.8491251
14 2004-04-01               1.5845274
15 2004-07-01               0.7469037
16 2004-10-01               0.6962047
17 2005-01-01               1.0488127
18 2005-04-01               0.9760271
19 2005-07-01               1.0272077
20 2005-10-01               1.0192201
21 2006-01-01               1.0509933
22 2006-04-01               1.2786165
23 2006-07-01               1.4850658
24 2006-10-01               2.0406845
25 2007-01-01               2.1389308
26 2007-04-0

## Preprocessing the predictors-  oecd inventory: oecd_inventory
This is a monthly data, we have to calculate the quaterly mean

In [64]:
# oecd_inventory = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'oecd_inventory', usecols= "A, B"))
oecd_inventory = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'oecd_inventory', usecols= "A, B"))
oecd_inventory = oecd_inventory.dropna()
oecd_inventory['Date'] = pd.to_datetime(oecd_inventory['Date'], format ="mixed")
oecd_inventory = oecd_inventory.set_index('Date')
oecd_inventory = oecd_inventory.resample('QS').mean()
oecd_inventory = oecd_inventory['2001-01-01' : '2023-06-30']
print(oecd_inventory)

            oecd_inventory
Date                      
2001-01-01     3819.207000
2001-04-01     3890.611667
2001-07-01     3929.645333
2001-10-01     3963.919000
2002-01-01     3962.906333
...                    ...
2019-10-01     4413.825366
2020-01-01     4456.016624
2020-04-01     4736.158983
2020-07-01     4763.088421
2020-10-01     4630.883993

[80 rows x 1 columns]


In [65]:
%%R -i oecd_inventory

oecd_inventory_r <- ts(oecd_inventory, start = 2001, end = 2024, frequency =4)
oecd_inventory_r <-  as.data.frame(as_tibble(seas(oecd_inventory_r, x11= "",)))
oecd_inventory_r <-  oecd_inventory_r[-c(3:7)]
colnames(oecd_inventory_r)[1] = "Date"
colnames(oecd_inventory_r)[2] = "oecd_inventory_sea"
print(oecd_inventory_r)

         Date oecd_inventory_sea
1  2001-01-01           3867.032
2  2001-04-01           3889.613
3  2001-07-01           3894.820
4  2001-10-01           3951.161
5  2002-01-01           4001.609
6  2002-04-01           3987.457
7  2002-07-01           3936.464
8  2002-10-01           3889.245
9  2003-01-01           3843.581
10 2003-04-01           3892.651
11 2003-07-01           3957.066
12 2003-10-01           3970.572
13 2004-01-01           3984.702
14 2004-04-01           3974.997
15 2004-07-01           4006.490
16 2004-10-01           4046.986
17 2005-01-01           4067.410
18 2005-04-01           4124.742
19 2005-07-01           4115.307
20 2005-10-01           4141.998
21 2006-01-01           4165.549
22 2006-04-01           4154.702
23 2006-07-01           4212.951
24 2006-10-01           4224.947
25 2007-01-01           4186.843
26 2007-04-01           4167.531
27 2007-07-01           4153.957
28 2007-10-01           4119.672
29 2008-01-01           4130.820
30 2008-04

## Preprocessing the predictors-  change in world oil consumption - del_world_consump
This is a quarterly data

In [66]:
# del_world_consump = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Data raw.xlsx', sheet_name = 'world_consumption', usecols= "A, B"))
del_world_consump = pd.DataFrame(pd.read_excel(r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Data raw.xlsx', sheet_name = 'world_consumption', usecols= "A, B"))
del_world_consump = del_world_consump.dropna()
del_world_consump['Date'] = pd.to_datetime(del_world_consump['Date'], format ="mixed")
del_world_consump = del_world_consump.set_index('Date')
del_world_consump.rename(columns = {"World Consumption Change": "del_world_consump",}, inplace = True)
del_world_consump = del_world_consump.sort_index().loc['2001-01-01' : '2023-06-30']
print(del_world_consump)

            del_world_consump
Date                         
2001-01-01               0.75
2001-04-01              -0.14
2001-07-01               0.63
2001-10-01               1.48
2002-01-01               2.52
...                       ...
2019-10-01              -9.22
2020-01-01              -7.87
2020-04-01               0.27
2020-07-01              11.34
2020-10-01               6.49

[80 rows x 1 columns]


In [67]:
%%R -i del_world_consump

del_world_consump_r <- ts(del_world_consump, start = 2001, end = 2024, frequency =4)
del_world_consump_r <-  as.data.frame(as_tibble(seas(del_world_consump_r, x11= "",)))
del_world_consump_r <-  del_world_consump_r[-c(3:7)]
colnames(del_world_consump_r)[1] = "Date"
colnames(del_world_consump_r)[2] = "del_world_consump_sea"
print(del_world_consump_r)

         Date del_world_consump_sea
1  2001-01-01            0.63106948
2  2001-04-01           -0.14464005
3  2001-07-01            0.66855274
4  2001-10-01            1.55499405
5  2002-01-01            2.42950049
6  2002-04-01            2.67258608
7  2002-07-01            1.58857552
8  2002-10-01            2.03383476
9  2003-01-01            2.04409311
10 2003-04-01            2.95459180
11 2003-07-01            4.82256474
12 2003-10-01            4.29377981
13 2004-01-01            3.72882561
14 2004-04-01            2.85417743
15 2004-07-01            2.01226790
16 2004-10-01            1.45993534
17 2005-01-01           -0.42327120
18 2005-04-01            0.63949749
19 2005-07-01            0.91893058
20 2005-10-01            1.51087520
21 2006-01-01            2.18825993
22 2006-04-01            1.23874903
23 2006-07-01            2.43914368
24 2006-10-01            1.73183275
25 2007-01-01            2.37054354
26 2007-04-01            0.67445525
27 2007-07-01            0.9

## Merging all the dataframes to create our dataset: seasonality unadjusted, python output

In [68]:
dat = pd.DataFrame(nm_oil_price.merge(wti_spot_price, on = 'Date').merge(brent_spot_price, on = 'Date').merge(dubai_price, on = 'Date')\
.merge(index_china_gdp, on = 'Date').merge(index_us_gdp, on = 'Date').merge(index_cboe_volt, on = 'Date')\
.merge(us_unemploy, on = 'Date').merge(index_ip_mining, on = 'Date').merge(index_cpi_china, on = 'Date').merge(two_x, on = 'Date').merge(index_cpi_us, on = 'Date')\
.merge(three_x, on = 'Date').merge(del_supply_nonopec, on = 'Date').merge(two_x_2, on = 'Date').merge(del_supply_saudi, on = 'Date')\
.merge(opec_spare_capacity, on = 'Date').merge(oecd_inventory, on = 'Date').merge(del_world_consump, on = 'Date'))

# file_name = r'C:\Users\asifr\OneDrive - State of New Mexico\Documents\Project Oil Price\Outputs\dat_pred_un_sea.csv'
file_name = r'C:\Users\asifr\OneDrive - The Pennsylvania State University\Documents\GitHub\Project-Oil-Price\Outputs\dat_pred_un_sea.csv'
dat.to_csv(file_name)

## Merging all the dataframes to create our dataset: seasonality adjusted, R output

In [69]:
%%R

dat_sea <- merge(nm_oil_price_r, wti_spot_price_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, brent_spot_price_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, dubai_price_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, index_china_gdp_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, index_us_gdp_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, index_cboe_volt_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, us_unemploy_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, index_ip_mining_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, index_cpi_china_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, two_x_opec_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, two_x_non_opec_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, index_cpi_us_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, three_x_index_energy_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, three_x_index_metal_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, three_x_index_precious_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, del_supply_nonopec_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, two_x_2_del_cap_per_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, two_x_2_del_world_gdp_per_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, del_supply_saudi_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, opec_spare_capacity_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, oecd_inventory_r, by = "Date",  all.x = TRUE)
dat_sea <- merge(dat_sea, del_world_consump_r, by = "Date",  all.x = TRUE)

# Using openxlsx
suppressPackageStartupMessages(library(openxlsx))

OUT <- createWorkbook()

## adding some worksheet to the workbook OUT

addWorksheet(OUT, "dat_pred_sea")

## Writing the data to the worksheets

writeData(OUT, sheet = "dat_pred_sea", x = dat_sea)

                      
# Exporting the file

# saveWorkbook(OUT, "C:/Users/asifr/OneDrive - State of New Mexico\Documents/Project Oil Price/Outputs/dat_pred_sea.xlsx", overwrite = TRUE)
saveWorkbook(OUT, "C:/Users/asifr/OneDrive - The Pennsylvania State University/Documents/GitHub/Project-Oil-Price/Outputs/dat_pred_sea.xlsx", overwrite = TRUE)

